# Lecture 4

**Authors:**
* Yilber Fabian Bautista
* Keiwan Jamaly

**Last date of modification:**
 November 28th 2021

Hello there, 

Welcome to Lecture 3 of this mini-lecture series on programing with Python. In this series, you will learn  basic and intermediate python tools that will be of great use in your scientific carer

**Objectives:** 

By the end of this lecture you will be able to:
* Understand the concept of solving differential equations
* solve differential equations using the scipy library's function **odeint**

# Theoretical background

Let's say, that you have a differential equation

\begin{equation}
\dot{x}(t) = -x(t)
\end{equation}

to compute $x(t+h)$ we can taylor expand around at $h=0$ for small $h$

\begin{equation}
x(t+h) = x(t) + \dot{x}(t)*h
\end{equation}

If we replace $\dot{x}(t)$ with $-x(t)$ from the differential equation we get
\begin{equation}
x(t+h) = x(t) - x(t)*h
\end{equation}

we can use this equation to compute a series of points for $x(t)$ for givin initial conditions. 

# Exercise 1

1. Compute solve the upper differential equation for $x(0)=1$ up to $t=1$ with $h=0.1$. 
2. Plot your result. 
3. Also compare the result with the analytical solution. 
4. What happens for different step sizes?

In [5]:
# Here goes your solution

What you have just done is called **Euler's method**. It can be generalized to any order of differential equations. 